In [ ]:
import logging
import sys

import numpy

import cicada.additive
import cicada.communicator
import time
from tqdm import *
from statistics import mean, stdev

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)#, modulus=251, precision=4)
    generator = numpy.random.default_rng(seed=1234)
    faster_times = []
    reg_times = []
    for i in tqdm(range(100)):
        #log.info("*" * 40, src=0)
        
        secret_share = protocol.uniform(generator=generator)
        secret = protocol.encoder.decode(protocol.reveal(secret_share))
        #log.info(f"Player {communicator.rank} secret: {secret}")

        t0f = time.time()
        floor_share_fast = protocol.floor_faster(secret_share)
        faster_times.append(time.time()-t0f)
        floor_fast = protocol.encoder.decode(protocol.reveal(floor_share_fast))
        
        t0r = time.time()
        floor_share_reg = protocol.floor_faster(secret_share)
        reg_times.append(time.time()-t0f)
        floor_reg = protocol.encoder.decode(protocol.reveal(floor_share_reg))
        
        if floor_reg != floor_fast:
            log.info(f"Player {communicator.rank} floor reg: {floor_reg} floor fast: {floor_fast}")
            
    log.info(f"Player {communicator.rank} floor reg time: {mean(reg_times)}s floor fast: {mean(faster_times)}s")

main();